# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [8]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# May take awhile locally
spark = SparkSession.builder.appName("Manipulate").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [2]:
path = "Datasets/"
tweets = spark.read.csv(path + "Rep_vs_Dem_tweets.csv", inferSchema=True, header=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [3]:
tweets.toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
...,...,...,...
92484,Republican,RepTomPrice,Check out my op-ed on need for End Executive O...
92485,Republican,RepTomPrice,"Yesterday, Betty &amp; I had a great time lear..."
92486,Republican,RepTomPrice,We are forever grateful for the service and sa...
92487,Republican,RepTomPrice,Happy first day of school @CobbSchools! #CobbB...


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [6]:
tweets.show(2, truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
only

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [7]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [18]:
latino = tweets.withColumn(
    "Latino_Mentions",
    F.regexp_extract(tweets["Tweet"], "(.*)(@LatinoLeader)(.*)", idx=2),
)
latino.limit(6).toPandas()

,Party,Handle,Tweet,Latino_Mentions
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,
3,"Congress has allocated about $18…""",None,None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,@LatinoLeader
5,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,


## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [24]:
# We haven't gotten to this yet so it's a bit of a teaser :)
counts = tweets.groupBy("Party").count()
counts.orderBy(F.desc("count")).show(6)

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
|               Today|   13|
+--------------------+-----+
only showing top 6 rows



In [25]:
clean_1 = (
    tweets.withColumn(
        "new_party",
        F.regexp_replace(tweets["Party"], "^(?!Democrat|Republican$).*", "Other"),
    )
    .filter("new_party='Other'")
    .limit(40)
    .toPandas()
)

In [41]:
from pyspark.sql.functions import when

clean = tweets.withColumn(
    "Party",
    when(tweets.Party == "Democrat", "Democrat")
    .when(tweets.Party == "Republican", "Republican")
    .otherwise("Other"),
)
counts = clean.groupBy("Party").count()
counts.orderBy(F.desc("count")).show(16)

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [42]:
print("OG Tweet")
tweets.select("tweet").show(1, False)

OG Tweet
+-------------------------------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L|
+-------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [46]:
# And here is the solution
print("Cleaned Tweet")
clean = clean.withColumn(
    "cleaned",
    F.regexp_replace(
        "Tweet",
        "(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?",
        "",
    ),
)
clean.show(1, False)

Cleaned Tweet
+--------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                      |cleaned                                                                                                             |
+--------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… 

## 4. Remove any leading or trailing white space in the tweet column

In [49]:
clean = clean.withColumn(
    "cleaned",
    F.trim(
        "Tweet",
    ),
)
clean.select("Tweet").show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

In [51]:
tweets.select("Tweet").show(5, False)
tweets.select("Tweet", F.trim(tweets.Tweet)).show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [52]:
renamed = tweets.withColumnRenamed("Party", "Dem_Rep")
renamed.limit(4).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [63]:
tweets.withColumn(
    "Concatenated", F.concat_ws(" ", *[tweets["Party"], tweets["Handle"]])
).show(2, truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|Party   |Handle       |Tweet                                                                                                                                       |Concatenated          |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|Democrat RepDarrenSoto|
+--------+-------------+-------------------------------

In [64]:
tweets.select(
    tweets.Party,
    tweets.Handle,
    F.concat_ws(" ", tweets.Party, tweets.Handle).alias("Concatenated"),
).show(5, False)

+----------------------------------+-------------+----------------------------------+
|Party                             |Handle       |Concatenated                      |
+----------------------------------+-------------+----------------------------------+
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Congress has allocated about $18…"|null         |Congress has allocated about $18…"|
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
+----------------------------------+-------------+----------------------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [65]:
# Parenthesis are used to mark a subexpression within a larger expression
# The . matches any character other than a new line
# | means is like or
# \w+ means followed by any word
pattern = "(.|" ")(#)(\w+)"
# * is used to match the preceding character zero or more times.
# ? will match the preceding character zero or one times, but no more.
# $ is used to match the ending position in a string.
split_pattern = r".*?({pattern})".format(pattern=pattern)
end_pattern = r"(.*{pattern}).*?$".format(pattern=pattern)

In [70]:
# $1 here means to capture the first part of the regex result
# The , will separate each find with a comma in the a array we create
df2 = tweets.withColumn("a", F.regexp_replace("Tweet", split_pattern, "$1,")).where(
    F.col("Tweet").like("%#%")
)
df2.select("a").show(3, False)
# Remove all the other results that came up
df3 = df2.withColumn("a", F.regexp_replace("a", end_pattern, "$1"))
df3.select("a").show(3, False)
# Finally create an array from the result by splitting on the comma
df4 = df3.withColumn("a", F.split("a", r","))
df4.select("a").show(3, False)
df4.limit(3).toPandas()

+-----------------------------------------------------------------------------------------+
|a                                                                                        |
+-----------------------------------------------------------------------------------------+
| #SaveTheInternet, #NetNeutrality, legislation here in the House… https://t.co/n3tggDLU1L|
| #NALCABPolicy2018,.…                                                                    |
| #NetNeutrality, rules. Find out…                                                        |
+-----------------------------------------------------------------------------------------+
only showing top 3 rows

+---------------------------------+
|a                                |
+---------------------------------+
| #SaveTheInternet, #NetNeutrality|
| #NALCABPolicy2018               |
| #NetNeutrality                  |
+---------------------------------+
only showing top 3 rows

+------------------------------------+
|a             

,Party,Handle,Tweet,a
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...","[ #SaveTheInternet, #NetNeutrality]"
1,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[ #NALCABPolicy2018]
2,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...,[ #NetNeutrality]


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [71]:
from pyspark.sql.types import *

md_office = [
    ("Mohammed", "Alfasy", "1987-4-8", "2016-1-7", "2017-2-3", "2018-3-2"),
    ("Marcy", "Wellmaker", "1986-4-8", "2015-1-7", "2017-1-3", "2018-1-2"),
    ("Ginny", "Ginger", "1986-7-10", "2014-8-7", "2015-2-3", "2016-3-2"),
    ("Vijay", "Doberson", "1988-5-2", "2016-1-7", "2018-2-3", "2018-3-2"),
    ("Orhan", "Gelicek", "1987-5-11", "2016-5-7", "2017-1-3", "2018-9-2"),
    ("Sarah", "Jones", "1956-7-6", "2016-4-7", "2017-8-3", "2018-10-2"),
    ("John", "Johnson", "2017-10-12", "2018-1-2", "2018-10-3", "2018-3-2"),
]

df = spark.createDataFrame(
    md_office, ["first_name", "last_name", "dob", "visit1", "visit2", "visit3"]
)  # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [72]:
# Covert the date columns into date types
df = (
    df.withColumn("dob", df["dob"].cast(DateType()))
    .withColumn("visit1", df["visit1"].cast(DateType()))
    .withColumn("visit2", df["visit2"].cast(DateType()))
    .withColumn("visit3", df["visit3"].cast(DateType()))
)

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+----------+----------+----------+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|
+----------+---------+----------+----------+----------+----------+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02|
|     Sarah|    Jones|1956-07-06|2016-04-07|2017-08-03|2018-10-02|
|      John|  Johnson|2017-10-12|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+----------+----------+----------+----------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [73]:
diff1 = df.select(F.datediff(df.visit2, df.visit1).alias("diff"))
diff2 = df.select(F.datediff(df.visit3, df.visit2).alias("diff"))

# Append the two dataframes together
diff_combo = diff1.union(diff2)
diff_combo.show(5)

+----+
|diff|
+----+
| 393|
| 727|
| 180|
| 758|
| 241|
+----+
only showing top 5 rows



## 8. Can you calculate the age of each patient?

In [ ]:
# We use the datediff function here as well
# And divide by 365 to get the age
# I also formated this value to get rid of all the decimal places
ages = df.select(format_number(datediff(df.visit1, df.dob) / 365, 1).alias("age"))
ages.show()

## 9. Can you extract the month from the first visit column and call it "Month"?

In [74]:
month1 = df.select(month(df["visit1"]).alias("Month"))
month1.show(3)

NameError: name 'month' is not defined

## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [75]:
path = "Datasets/"
sales = spark.read.csv(path + "supermarket_sales.csv", inferSchema=True, header=True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [76]:
sales.limit(6).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,2022-10-09 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,2022-10-09 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,2022-10-09 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,2022-10-09 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,2022-10-09 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,2022-10-09 18:30:00,Ewallet,597.73,4.761905,29.8865,4.1


In [78]:
sales.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [81]:
print("This gives all null values")
df = sales.withColumn("Formatted Date", sales["Date"].cast(DateType()))
df = df.select("Date", "Formatted Date")
print(df.limit(6).toPandas())

print(" ")
print("This result gives the wrong results (notice that all months are january)")
sales.select(
    "Date",
    F.to_date(sales.Date, "mm/dd/yyyy").alias("Dateformatted"),
    F.month(F.to_date(sales.Date, "mm/dd/yyyy")).alias("Month"),
).show(5)


# So I will try to get creative with my solultions below

This gives all null values
        Date Formatted Date
0   1/5/2019           None
1   3/8/2019           None
2   3/3/2019           None
3  1/27/2019           None
4   2/8/2019           None
5  3/25/2019           None
 
This result gives the wrong results (notice that all months are january)
22/10/09 02:11:14 ERROR Executor: Exception in task 0.0 in stage 73.0 (TID 103)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/5/2019' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1066)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTime

Py4JJavaError: An error occurred while calling o655.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 73.0 failed 1 times, most recent failure: Lost task 0.0 in stage 73.0 (TID 103) (192.168.7.139 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/5/2019' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1066)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.Abstract

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [83]:
# We need to creative here
# First split the date field and get the month value
df = sales.select("Date", F.split(sales.Date, "/")[0].alias("Month"), "Total")

# Verify everything worked correctly
print("Verify")
df.show(5)
print(df.printSchema())

Verify
+---------+-----+--------+
|     Date|Month|   Total|
+---------+-----+--------+
| 1/5/2019|    1|548.9715|
| 3/8/2019|    3|   80.22|
| 3/3/2019|    3|340.5255|
|1/27/2019|    1| 489.048|
| 2/8/2019|    2|634.3785|
+---------+-----+--------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Total: double (nullable = true)

None


## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [85]:
values = [
    (5, "Epic. This is the best movie I have EVER seen"),
    (4, "Pretty good, but I would have liked to seen better special effects"),
    (3, "So so. Casting could have been improved"),
    (
        5,
        "The most EPIC movie of the year! Casting was awesome. Special effects were so intense.",
    ),
    (4, "Solid but I would have liked to see more of the love story"),
    (5, "THE BOMB!!!!!!!"),
]
reviews = spark.createDataFrame(values, ["rating", "review_txt"])

reviews.show(6, False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [88]:
# We can do 1-3 in one call here
df = reviews.withColumn(
    "cleaned_reviews",
    F.trim(F.lower(F.regexp_replace(F.col("review_txt"), "[^\sa-zA-Z0-9]", ""))),
)
df.show(1, False)

+------+---------------------------------------------+--------------------------------------------+
|rating|review_txt                                   |cleaned_reviews                             |
+------+---------------------------------------------+--------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen|epic this is the best movie i have ever seen|
+------+---------------------------------------------+--------------------------------------------+
only showing top 1 row



In [90]:
# Then split on the spaces!
df = df.withColumn("review_txt_array", F.split(F.col("cleaned_reviews"), " "))
df.show(1, False)

+------+---------------------------------------------+--------------------------------------------+-------------------------------------------------------+
|rating|review_txt                                   |cleaned_reviews                             |review_txt_array                                       |
+------+---------------------------------------------+--------------------------------------------+-------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen|epic this is the best movie i have ever seen|[epic, this, is, the, best, movie, i, have, ever, seen]|
+------+---------------------------------------------+--------------------------------------------+-------------------------------------------------------+
only showing top 1 row



## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [93]:
epic = df.withColumn("result", F.array_contains(F.col("review_txt_array"), "epic"))
epic.toPandas()

,rating,review_txt,cleaned_reviews,review_txt_array,result
0,5,Epic. This is the best movie I have EVER seen,epic this is the best movie i have ever seen,"[epic, this, is, the, best, movie, i, have, ev...",True
1,4,"Pretty good, but I would have liked to seen be...",pretty good but i would have liked to seen bet...,"[pretty, good, but, i, would, have, liked, to,...",False
2,3,So so. Casting could have been improved,so so casting could have been improved,"[so, so, casting, could, have, been, improved]",False
3,5,The most EPIC movie of the year! Casting was a...,the most epic movie of the year casting was aw...,"[the, most, epic, movie, of, the, year, castin...",True
4,4,Solid but I would have liked to see more of th...,solid but i would have liked to see more of th...,"[solid, but, i, would, have, liked, to, see, m...",False
5,5,THE BOMB!!!!!!!,the bomb,"[the, bomb]",False


### That's it! Great Job!